# Prediction notebook

----------------------
This notebook allows for predictions on large images. It includes the tiling of those images either without smooth blending or with smooth blending

To do for later use:

- Smooth blending only works on very small images. Why? Can we improve?
- The predict function does currently not work with pretrained versions of the model
- Rewrite so that it automatically predicts on all images
- Second plot to also compare with mask 

In [ ]:
from patchify import patchify, unpatchify
import cv2
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model
import os
import random
from keras.metrics import MeanIoU

In [ ]:
#Load model
model = load_model('../models/StdUnet_diceplusfocal_epochs100_batchsize8_learningrate0.001.hdf5', compile=False)

In [ ]:
#Load all test images and masks
test_img_dir = '../data/data_original/test_data/image/'
test_msk_dir = '../data/data_original/test_data/mask/'

img_list = os.listdir(test_img_dir)
img_list.sort()
msk_list = os.listdir(test_msk_dir)
msk_list.sort()
num_images = len(os.listdir(test_img_dir))

#### Test on a single image

In [ ]:
# Load one random quarter for fast testing
img_num = random.randint(0, num_images-1)
img_for_test = cv2.imread(test_img_dir+img_list[img_num], 0)
mask_for_test =cv2.imread(test_msk_dir+msk_list[img_num], 0)

In [ ]:
#This will split the image into small images of shape [x,y]
patches = patchify(img_for_test, (512, 512), step=512) 
print("Large image shape is: ", img_for_test.shape)
print("Patches array shape is: ", patches.shape)

In [ ]:
patches_mask = patchify(mask_for_test, (512, 512), step=512) 
print("Large image shape is: ", img_for_test.shape)
print("Patches array shape is: ", patches.shape)

In [ ]:
plt.figure(figsize=(9, 9))
square = 6
ix = 1
for i in range(square):
	for j in range(square):
		# specify subplot and turn of axis
		ax = plt.subplot(square, square, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot 
		plt.imshow(patches[i, j, :, :], cmap='gray')
		ix += 1
# show the figure
plt.show()

In [ ]:
def dice_metric(y_pred, y_true):
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    # if y_pred.sum() == 0 and y_pred.sum() == 0:
    #     return 1.0

    return 2*intersection / union

In [ ]:
# Make predictions and save predictions

predicted_patches = []
mask_patches = []

pred_patches = []
true_patches = []

for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        print(i)
        single_patch = patches[i,j,:,:]
        single_mask = patches_mask[i,j,:,:]
        
        single_patch = single_patch / 255.
        single_mask = single_mask / 255.

        single_patch = np.expand_dims(np.array(single_patch), axis=2)
        single_mask = np.expand_dims(np.array(single_mask), axis=2)
        
        single_patch_input=np.expand_dims(single_patch, 0)
        single_mask_input=np.expand_dims(single_mask, 0)

        single_patch_prediction = (model.predict(single_patch_input))

        single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]
        single_mask_predict = np.argmax(single_mask_input, axis=3)[0,:,:]

        predicted_patches.append(single_patch_predicted_img)
        mask_patches.append(single_mask_predict)

        #flatten the predict and the true mask
        pred = single_patch_predicted_img.flatten()
        true = single_mask_predict.flatten()

        pred_patches.append(pred)
        true_patches.append(true)


In [ ]:
mean_IoU = []
class_1_IoU = []
class_2_IoU = []
class_3_IoU = []
class_4_IoU = []





for k in range(len(pred)):
    #instantiate true-false classes
    c1_true = 0
    c1_false = 0
    c2_true = 0
    c2_false = 0
    c3_true = 0
    c3_false = 0
    c4_true = 0
    c4_false = 0

    all_true = 0
    all_false = 0
    # Class 1 (background)
    if true[k] == 0:
        if pred[k] == 0:
            all_true+=1
            c1_true+=1
        else:
            all_false+=1
            c1_false+=1
    # Class 2 (fractures)
    elif true[k] == 1:
        if pred[k] == 1:
            all_true+=1
            c2_true+=1
        else:
            all_false+=1
            c2_false+=1
    # Class 3 (pore)
    elif true[k] == 2:
        if pred[k] == 2:
            all_true+=1
            c3_true+=1
        else:
            all_false+=1
            c3_false+=1
    else:
        if pred[k] == 3:
            all_true+=1
            c4_true+=1
        else:
            all_false+=1
            c4_false+=1

mean_IoU.append(all_true/(all_true+all_false))

#------------------------------------------------------#


In [ ]:
# can be added to the end of the above function, works kinda but not really
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(single_mask_predict, single_patch_predicted_img)
mean_IoU.append(IOU_keras.result().numpy())
########
predicted_patches.append(single_patch_predicted_img)
mask_patches.append(single_mask_predict)

values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

class_1.append(class1_IoU)
class_2.append(class2_IoU)
class_3.append(class3_IoU)
class_4.append(class4_IoU)

In [ ]:
predicted_patches = np.array(predicted_patches)
predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 512,512)) #Replace with patch size

masks = np.array(mask_patches)
mask_reshaped = np.reshape(masks, (patches.shape[0], patches.shape[1], 512,512))

In [ ]:
plt.figure(figsize=(9, 9))
square_1 = 6
square_2 = 8
ix = 1
for i in range(square_1):
	for j in range(square_2):
		# specify subplot and turn of axis
		ax = plt.subplot(square_1, square_2, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot 
		plt.imshow(predicted_patches_reshaped[i, j, :, :], cmap='gray')
		ix += 1
# show the figure
plt.show()

In [ ]:
reconstructed_image = unpatchify(predicted_patches_reshaped, (3072, 4096))
reconstructed_mask = unpatchify(mask_reshaped, (3072, 4096))

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(221)
plt.title('Original Image')
plt.imshow(img_for_test, cmap='gray')
plt.subplot(222)
plt.title('Prediction')
plt.imshow(reconstructed_image, cmap = 'gray')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(221)
plt.title('Original Mask')
plt.imshow(mask_for_test, cmap='gray')
plt.subplot(222)
plt.title('Prediction')
plt.imshow(reconstructed_image, cmap = 'gray')
plt.show()

### Predictions with IoU

In [ ]:
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(reconstructed_mask, reconstructed_image)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)

### Predict on large image -- Baustelle

In [ ]:
large_image_scaled = img_for_test /255.
large_image_scaled = np.expand_dims(large_image_scaled, axis=2)

large_image_scaled.shape


In [ ]:
scale_percent = 10 # percent of original size
width = int(large_image_scaled.shape[1] * scale_percent / 100)
height = int(large_image_scaled.shape[0] * scale_percent / 100)
dim = (width, height)

In [ ]:
smaller_image = cv2.resize(large_image_scaled, dim, interpolation=cv2.INTER_AREA)

In [ ]:
smaller_image.shape

In [ ]:
patch_size=512
n_classes=4

In [ ]:
# Use the algorithm. The `pred_func` is passed and will process all the image 8-fold by tiling small patches with overlap, called once with all those image as a batch outer dimension.
# Note that model.predict(...) accepts a 4D tensor of shape (batch, x, y, nb_channels), such as a Keras model.
from smooth_tiled_pred import predict_img_with_smooth_windowing

predictions_smooth = predict_img_with_smooth_windowing(
    large_image_scaled,    #Must be of shape (x, y, c) --> NOT of the shape (n, x, y, c)
    window_size=patch_size,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=n_classes,
    pred_func=(
        lambda img_batch_subdiv: model.predict((img_batch_subdiv))
    )
)

In [ ]:
final_prediction = np.argmax(predictions_smooth, axis=2)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(131)
plt.title('Testing Image')
plt.imshow(img_for_test, cmap='gray')
plt.subplot(132)
plt.title('Prediction without smooth blending')
plt.imshow(reconstructed_image)
plt.subplot(133)
plt.title('Prediction with smooth blending')
plt.imshow(final_prediction)
plt.show()